In [93]:
import pandas as pd
import datetime as dt 
import numpy as np
import matplotlib.pyplot as plt

cust_data = pd.read_csv("/home/labsuser/Datasets/Comcast_telecom_complaints_data.csv")
cust_data.head(10)
cust_data['Date'] = pd.to_datetime(cust_data['Date'],format="%d-%m-%y")
cust_data['Month'] = cust_data['Date'].dt.month

#Month wise data trend chart

plt.plot(cust_data.groupby(['Month']).count()['Ticket #'],label="Count of Tickets",marker='o')
plt.xlabel('Month')
plt.ylabel('Count')
plt.xticks(np.arange(0,14,2))
plt.legend()

#Date wise data trend chart
plt.figure(0)
plt.figure(figsize=(100,10))
plt.plot(cust_data.groupby(['Date_month_year']).count()['Ticket #'],label="Count of Tickets daily",marker='o')
plt.xlabel('Date')
plt.ylabel('Count')
plt.legend()
plt.savefig("DateChart.jpg")

print("Grouped_data-> \n",cust_data.groupby(['Customer Complaint']).count().sort_values('Ticket #',ascending=False)['Ticket #'])
print("As per the counts most of the complaints are of the category - Comcast. Some of them are related to Comcast internet and even billing.")




#Creating new Status variable for Open and Closed Complaints
cust_data.loc[cust_data['Status'].str.contains('Open') | cust_data['Status'].str.contains('Pending'), 'Cstatus'] = 'Open'
cust_data.loc[cust_data['Status'].str.contains('Solved') | cust_data['Status'].str.contains('Closed'), 'Cstatus'] = 'Closed'
#cust_data[['Cstatus','Status']].head(10)

#Statewise Open and closed complaints bar graph. Blue -> Open and Orange -> Closed
aggregated = pd.crosstab(cust_data['State'],cust_data['Cstatus'])
aggregated_new = pd.DataFrame(aggregated)
#print("Aggregated Data -> \n",aggregated_new.head(10))
plt.figure(figsize=(100,30))
plt.bar(aggregated_new.index,aggregated_new.Open,label='Open')
plt.bar(aggregated_new.index,aggregated_new.Closed,bottom=aggregated_new.Open,label='Closed')
plt.legend()
plt.savefig("StackedBarChart.jpg")

aggregated_new['Total'] = aggregated_new.sum(axis=1)
aggregated_new['Unresolved Complaints %'] = (aggregated_new.Open/aggregated_new.Total) *100

print("Aggregated Data with Total and Percentage data -> \n",aggregated_new.head(10))
highest_comp = aggregated_new[aggregated_new.Total == aggregated_new.Total.max()]['Total']
print("State with max number of complaints : ", aggregated_new[aggregated_new.Total == aggregated_new.Total.max()].index.values,"\nNumber of complaints -> \n",highest_comp)
highest_perc = aggregated_new[aggregated_new['Unresolved Complaints %'] == aggregated_new['Unresolved Complaints %'].max()]['Unresolved Complaints %']
print("State with max percentage of unresolved complaints  : ", aggregated_new[aggregated_new['Unresolved Complaints %'] == aggregated_new['Unresolved Complaints %'].max()].index.values, "\nPercentage -> \n",highest_perc )



aggregated_recvd_via = pd.crosstab(cust_data['Received Via'],cust_data['Cstatus'])
aggregated_recvd = pd.DataFrame(aggregated_recvd_via)
aggregated_recvd['Total'] = aggregated_recvd_via.sum(axis=1)
aggregated_recvd['Resolved Complaints %'] = (aggregated_recvd['Closed']/aggregated_recvd['Total']) *100
print("Aggregated Data with total & Percentage columns ->\n",aggregated_recvd)

print("Percentage of Closed complaints via - Customer Care Call is - " + str(int(aggregated_recvd.loc['Customer Care Call']['Resolved Complaints %'])) + "%")
print("Percentage of Closed complaints via - Internet is - " + str(int(aggregated_recvd.loc['Internet']['Resolved Complaints %'])) + "%")



Aggregated Data with total & Percentage columns ->
 Cstatus             Closed  Open  Total  Resolved Complaints %
Received Via                                                  
Customer Care Call     864   255   1119              77.211796
Internet               843   262   1105              76.289593
Percentage of Closed complaints via - Customer Care Call is - 77%
Percentage of Closed complaints via - Internet is - 76%
